In [1]:
from utils import *

Setup_environment()

from hpe_wrapper import Wrapper_2Dpose, Wrapper_3Dpose

model_2D ='./detectron/configs/COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml'
weights_2D = './detectron/checkpoint/model_final_997cc7.pkl'
model_3D = './VideoPose3D/checkpoint/pretrained_h36m_detectron_coco.bin'

pose2d = Wrapper_2Dpose(model=model_2D, weights= weights_2D , ROI_thr=0.75)
pose_3d = Wrapper_3Dpose(model_3D)

config  = load_config()
print(config['video_object'])

video_object = Video_wrapper(config['video_object'], resize_video_by=0.3) #, start=0, end=10) # downsample_fps_by=2,
data_2d, metadata_vid = pose2d.predict_2D_poses(input_video_object=video_object)
data_3d = pose_3d.predict_3D_poses(data_2d, metadata_vid)

./data/videos/remastered_omco_pickingup_1 (video-converter.com).mp4
Frame 0 processed in 1.119s
Frame 1 processed in 0.101s
Frame 2 processed in 0.129s
Frame 3 processed in 0.070s
Frame 4 processed in 0.069s
Frame 5 processed in 0.079s
Frame 6 processed in 0.069s
Frame 7 processed in 0.066s
Frame 8 processed in 0.068s
Frame 9 processed in 0.069s
Frame 10 processed in 0.061s
Frame 11 processed in 0.064s
Frame 12 processed in 0.059s
Frame 13 processed in 0.059s
Frame 14 processed in 0.059s
Frame 15 processed in 0.061s
Frame 16 processed in 0.063s
Frame 17 processed in 0.060s
Frame 18 processed in 0.062s
Frame 19 processed in 0.059s
Frame 20 processed in 0.060s
Frame 21 processed in 0.061s
Frame 22 processed in 0.062s
Frame 23 processed in 0.062s
Frame 24 processed in 0.061s
Frame 25 processed in 0.058s
Frame 26 processed in 0.058s
Frame 27 processed in 0.058s
Frame 28 processed in 0.059s
Frame 29 processed in 0.058s
Frame 30 processed in 0.058s
Frame 31 processed in 0.058s
Frame 32 proce

## Pose Optim  Loop

In [2]:
import torch
from tensor_ergonomics import RULAXXX

def run_pose_optimization(original_pose, num_steps=3):
    """
    Optimize the pose using the RULA score.
    
    Parameters:
    - original_pose: A tensor of shape (17, 3)
    
    Returns:
    - Optimized pose.
    """
    torch.autograd.set_detect_anomaly(True)
    if not isinstance(original_pose, torch.Tensor):
        original_pose = torch.tensor(original_pose)

    original_pose = original_pose.clone().detach().requires_grad_(True)

    optimizer = torch.optim.Adam([original_pose], lr=0.1) #(1,200t, 17, 3)

    for step in range(num_steps):
        optimizer.zero_grad()

        rula_eval = RULAXXX(original_pose)
        rula_score = rula_eval.compute_scores()
        print(rula_score.grad_fn)
        ergo_sum = rula_score.sum()
        loss = ergo_sum

        loss.backward()
        print(original_pose.grad)
        optimizer.step()
        print(f"Step {step}: Loss: {loss.item()}, RULA Score: {rula_score.sum().item()}")
        
    return original_pose

optimized_pose = run_pose_optimization(data_3d)

p = optimized_pose.detach().numpy()
print('Original pose: ', data_3d[1,1,])
print('Optim pose: ', p[1,1,:])

RuntimeError: Tensors must have same number of dimensions: got 2 and 1

In [ ]:
stop

In [ ]:
import plotly.graph_objects as go
import webbrowser
import time

def visualize_frame(frame):
    # Extract x, y, z coordinates of keypoints
    x = frame[:, 0]
    y = frame[:, 2]
    z = - frame[:, 1]

    # Create a scatter plot for the keypoints
    scatter = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(size=5, color='red')
    )

    # Define pairs of keypoints to connect with green lines
    pairs = [(0,1), (0,4), (0,7), (7,8), (8,9), (9,10), (4,5), (1,2), (5,6), (2,3),
             (8,11), (8,14), (11,12), (14,15), (12,13), (15,16)]

    lines = []
    for pair in pairs:
        lines.append(
            go.Scatter3d(
                x=[x[pair[0]], x[pair[1]]],
                y=[y[pair[0]], y[pair[1]]],
                z=[z[pair[0]], z[pair[1]]],
                mode='lines',
                line=dict(color='green', width=5)
            )
        )

    # Create the layout and add all traces
    layout = go.Layout(
        scene=dict(
            xaxis=dict(nticks=10, range=[min(x), max(x)]),
            yaxis=dict(nticks=10, range=[min(y), max(y)]),
            zaxis=dict(nticks=10, range=[min(z), max(z)])
        )
    )

    fig = go.Figure(data=[scatter] + lines, layout=layout)

    # Save the plot as an HTML file
    filename = "plot.html"
    fig.write_html(filename)

    # Open the saved HTML file in the default web browser
    webbrowser.open(filename)
    time.sleep(3)

# Example usage
# Assuming data_3d is your 3D array of shape (199, 17, 3)
for frame in p[9:10,:,:]:
    visualize_frame(frame)

In [ ]:
import torch
from tensor_ergonomics import RULAXXX

def run_pose_optimization(original_pose, num_steps=300):
    """
    Optimize the pose using the RULA score.
    
    Parameters:
    - original_pose: A tensor of shape (17, 3)
    
    Returns:
    - Optimized pose.
    """

    # Initialize the pose to be optimized
    pose = original_pose.clone().detach().requires_grad_(True)

    # Define an optimizer. Here, we use the Adam optimizer.
    optimizer = torch.optim.Adam([pose], lr=0.01)

    for step in range(num_steps):
        optimizer.zero_grad()

        # Compute the RULA score for the current pose            
        rula_eval = RULAXXX(original_pose)
        print(rula_eval)
        rula_score = rula_eval.compute_scores()
        
        # We want to minimize the RULA score, so we negate it for optimization
        loss = -rula_score

        # Backpropagate the loss
        loss.backward()

        # Update the pose
        optimizer.step()

        # Print the RULA score every 50 steps
        if step % 50 == 0:
            print(f"Step {step}: RULA Score: {rula_score.item()}")

    return pose


# Optimize the pose
original_pose = data_3d
original_pose_tensor = torch.tensor(original_pose, dtype=torch.float32, requires_grad=True)
print(original_pose_tensor.shape) #torch.Size([17, 3])
optimized_pose = run_pose_optimization(original_pose_tensor)
